In [22]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10
import tensorflow as tf
# Network Parameters
num_hidden_1 = 256 # 1st layer num features
num_hidden_2 = 128
num_hidden_3 = 64
num_hidden_4 = 32# 2nd layer num features (the latent dim)
num_input = 39 
num_classes = 7

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'encoder_h3': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_3])),
    'encoder_h4': tf.Variable(tf.random_normal([num_hidden_3, num_hidden_4])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_4, num_hidden_3])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_3, num_hidden_2])),
    'decoder_h3': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h4': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
    'classifier_h': tf.Variable(tf.random_normal([num_hidden_4, num_classes])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'encoder_b3': tf.Variable(tf.random_normal([num_hidden_3])),
    'encoder_b4': tf.Variable(tf.random_normal([num_hidden_4])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_3])),
    'decoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b3': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b4': tf.Variable(tf.random_normal([num_input])),
    'classifier_b': tf.Variable(tf.random_normal([num_classes])),
}

In [23]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['encoder_h3']),
                                   biases['encoder_b3']))
    layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['encoder_h4']),
                                   biases['encoder_b4']))
    
    return layer_4


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['decoder_h3']),
                                   biases['decoder_b3']))
    layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['decoder_h4']),
                                   biases['decoder_b4']))
    return layer_4

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op

label_pred = tf.nn.softmax(tf.add(tf.matmul(encoder_op, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [24]:
from numpy import genfromtxt
my_data = genfromtxt('./DataForBiplab/2017Kharif_NDVI1.csv', delimiter=',')
my_data1 = genfromtxt('./DataForBiplab/2017Kharif_VV1.csv', delimiter=',')

In [25]:
data = my_data[1:,2:]
data1 = my_data1[1:,2:]
print(data.shape)
print(data1.shape)

(546, 13)
(546, 14)


In [26]:
for i in range (0,data1.shape[0]):
    for j in range (0,data1.shape[1]):
        if (data1[i][j]>1):
            print(data1[i][j])

1.26887548
1.053582072
1.302524686
1.063615799
2.199306011
3.165250778
1.117632627
1.054525018
1.730545163


In [27]:
print(data.max())
print(data1.max())

0.855952701
3.165250778


In [28]:

import numpy as np
data = np.array(data)
data1 = np.array(data1)
print(data.shape)
print(data1.shape)

(546, 13)
(546, 14)


In [29]:
import pandas
colnames = ['Latitude', 'Longitude', 'CropType','SampleYear','ShowingPeriod'
           , 'HarvestingPeriod', 'Others']
data3 = pandas.read_csv('./DataForBiplab/FieldDataKharif2017.csv', names=colnames)

In [30]:
ctype = data3.CropType.tolist()
print(ctype)

['CropType', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcan

In [31]:
p = np.array(ctype)
print(p.shape)

(547,)


In [32]:
lbl1 = ctype
label = []
for i in range(1,547):
    if(lbl1[i] == "Paddy"):
        label += [0]
    if(lbl1[i] == "Sugarcane"):
        label += [1]
    if(lbl1[i] == "PearlMillet"):
        label += [2]
    if(lbl1[i] == "PigeonPea"):
        label += [3]
    if(lbl1[i] == "Sorghum"):
        label += [4]
    if(lbl1[i] == "Forest/NonCropland"):
        label += [5]
    if(lbl1[i] == "Habitation"):
        label += [6]
    
print(label)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 

In [33]:
label = np.array(label)
print(label.shape)

(546,)


In [34]:
print(data.shape)
print(data1.shape)
data2 = np.concatenate((data,data1), axis=1)
print(data2.shape)

(546, 13)
(546, 14)
(546, 27)


In [46]:
x1=[]
x2=[]
x3=[]
x4=[]
x5=[]
x6=[]
x7=[]
l1=[]
l2=[]
l3=[]
l4=[]
l5=[]
l6=[]
l7=[]
for i in range(0, label.shape[0]):
    if(label[i]==0):
        x1.append(data2[i])
        l1.append([1,0,0,0,0,0,0])
    if(label[i]==1):
        x2.append(data2[i])
        l2.append([0,1,0,0,0,0,0])
    if(label[i]==2):
        x3.append(data2[i])
        l3.append([0,0,1,0,0,0,0])
    if(label[i]==3):
        x4.append(data2[i])
        l4.append([0,0,0,1,0,0,0])
    if(label[i]==4):
        x5.append(data2[i])
        l5.append([0,0,0,0,1,0,0])
    if(label[i]==5):
        x6.append(data2[i])
        l6.append([0,0,0,0,0,1,0])
    if(label[i]==6):
        x7.append(data2[i])
        l7.append([0,0,0,0,0,0,1])
x1=np.array(x1)
x2=np.array(x2)
x3=np.array(x3)
x4=np.array(x4)
x5=np.array(x5)
x6=np.array(x6)
x7=np.array(x7)

l1=np.array(l1)
l2=np.array(l2)
l3=np.array(l3)
l4=np.array(l4)
l5=np.array(l5)
l6=np.array(l6)
l7=np.array(l7)

print(l7.shape)
print(x6.shape)

(209, 7)
(34, 27)


In [36]:
from sklearn.utils import shuffle
data2, label = shuffle(data2, label, random_state = 0)

In [37]:
# 80% train 20% test (NDVIclean)
X_train = data2[0:436,:]
X_test = data2[436:,:]
y_train = label[0:436]
y_test = label[436:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(436, 27)
(110, 27)
(436,)
(110,)


In [38]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(436, 7)
(110, 7)


In [39]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 20
num_batch = 12

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.78480228
Epoch: 2 cost = 0.75620715
Epoch: 3 cost = 0.72872428
Epoch: 4 cost = 0.70206322
Epoch: 5 cost = 0.67611559
Epoch: 6 cost = 0.65097868
Epoch: 7 cost = 0.62694703
Epoch: 8 cost = 0.60444895
Epoch: 9 cost = 0.58393270
Epoch: 10 cost = 0.56573643
Epoch: 11 cost = 0.54998382
Epoch: 12 cost = 0.53654786
Epoch: 13 cost = 0.52510625
Epoch: 14 cost = 0.51525923
Epoch: 15 cost = 0.50663793
Epoch: 16 cost = 0.49895491
Epoch: 17 cost = 0.49200203
Epoch: 18 cost = 0.48562954
Epoch: 19 cost = 0.47972681
Epoch: 20 cost = 0.47421036
Epoch: 21 cost = 0.46901615
Epoch: 22 cost = 0.46409311
Epoch: 23 cost = 0.45939855
Epoch: 24 cost = 0.45489373
Epoch: 25 cost = 0.45054203
Epoch: 26 cost = 0.44630835
Epoch: 27 cost = 0.44215965
Epoch: 28 cost = 0.43806632
Epoch: 29 cost = 0.43400298
Epoch: 30 cost = 0.42994867
Epoch: 31 cost = 0.42588605
Epoch: 32 cost = 0.42179944
Epoch: 33 cost = 0.41767253
Epoch: 34 cost = 0.41348573
Epoch: 35 cost = 0.40921354
Epoch: 36 cost = 0.40482233
E

Epoch: 294 cost = 0.00081804
Epoch: 295 cost = 0.00093939
Epoch: 296 cost = 0.00084372
Epoch: 297 cost = 0.00103190
Epoch: 298 cost = 0.00113299
Epoch: 299 cost = 0.00078171
Epoch: 300 cost = 0.00069226
Epoch: 301 cost = 0.00071019
Epoch: 302 cost = 0.00072066
Epoch: 303 cost = 0.00077365
Epoch: 304 cost = 0.00076443
Epoch: 305 cost = 0.00066052
Epoch: 306 cost = 0.00076488
Epoch: 307 cost = 0.00094056
Epoch: 308 cost = 0.00106651
Epoch: 309 cost = 0.00087955
Epoch: 310 cost = 0.00079115
Epoch: 311 cost = 0.00074128
Epoch: 312 cost = 0.00069173
Epoch: 313 cost = 0.00068585
Epoch: 314 cost = 0.00063708
Epoch: 315 cost = 0.00072577
Epoch: 316 cost = 0.00093698
Epoch: 317 cost = 0.00085118
Epoch: 318 cost = 0.00082759
Epoch: 319 cost = 0.00078064
Epoch: 320 cost = 0.00079628
Epoch: 321 cost = 0.00092319
Epoch: 322 cost = 0.00066617
Epoch: 323 cost = 0.00059513
Epoch: 324 cost = 0.00061187
Epoch: 325 cost = 0.00068465
Epoch: 326 cost = 0.00078060
Epoch: 327 cost = 0.00080180
Epoch: 328 cos

Epoch: 586 cost = 0.00040710
Epoch: 587 cost = 0.00035862
Epoch: 588 cost = 0.00048928
Epoch: 589 cost = 0.00041725
Epoch: 590 cost = 0.00044925
Epoch: 591 cost = 0.00051576
Epoch: 592 cost = 0.00043068
Epoch: 593 cost = 0.00037126
Epoch: 594 cost = 0.00038887
Epoch: 595 cost = 0.00041913
Epoch: 596 cost = 0.00033680
Epoch: 597 cost = 0.00040820
Epoch: 598 cost = 0.00050468
Epoch: 599 cost = 0.00042586
Epoch: 600 cost = 0.00035292
Epoch: 601 cost = 0.00043823
Epoch: 602 cost = 0.00044488
Epoch: 603 cost = 0.00036439
Epoch: 604 cost = 0.00036413
Epoch: 605 cost = 0.00045371
Epoch: 606 cost = 0.00050805
Epoch: 607 cost = 0.00033487
Epoch: 608 cost = 0.00036592
Epoch: 609 cost = 0.00043809
Epoch: 610 cost = 0.00040062
Epoch: 611 cost = 0.00036553
Epoch: 612 cost = 0.00045004
Epoch: 613 cost = 0.00052699
Epoch: 614 cost = 0.00035041
Epoch: 615 cost = 0.00033044
Epoch: 616 cost = 0.00031728
Epoch: 617 cost = 0.00032805
Epoch: 618 cost = 0.00039648
Epoch: 619 cost = 0.00046475
Epoch: 620 cos

Epoch: 882 cost = 0.00027107
Epoch: 883 cost = 0.00025332
Epoch: 884 cost = 0.00038824
Epoch: 885 cost = 0.00034198
Epoch: 886 cost = 0.00024395
Epoch: 887 cost = 0.00024347
Epoch: 888 cost = 0.00038391
Epoch: 889 cost = 0.00042407
Epoch: 890 cost = 0.00027988
Epoch: 891 cost = 0.00031649
Epoch: 892 cost = 0.00030135
Epoch: 893 cost = 0.00024517
Epoch: 894 cost = 0.00027794
Epoch: 895 cost = 0.00027760
Epoch: 896 cost = 0.00025617
Epoch: 897 cost = 0.00031205
Epoch: 898 cost = 0.00033133
Epoch: 899 cost = 0.00035026
Epoch: 900 cost = 0.00039911
Epoch: 901 cost = 0.00041323
Epoch: 902 cost = 0.00025430
Epoch: 903 cost = 0.00029961
Epoch: 904 cost = 0.00037232
Epoch: 905 cost = 0.00027105
Epoch: 906 cost = 0.00024835
Epoch: 907 cost = 0.00028130
Epoch: 908 cost = 0.00028835
Epoch: 909 cost = 0.00029870
Epoch: 910 cost = 0.00027046
Epoch: 911 cost = 0.00039333
Epoch: 912 cost = 0.00039497
Epoch: 913 cost = 0.00028374
Epoch: 914 cost = 0.00031595
Epoch: 915 cost = 0.00027691
Epoch: 916 cos

In [40]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [41]:
print(y_test.shape)

(110, 7)


In [42]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.78181815


In [47]:
res1=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x1, Y: l1}))
res2=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x2, Y: l2}))
res3=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x3, Y: l3}))
res4=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x4, Y: l4}))
res5=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x5, Y: l5}))
res6=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x6, Y: l6}))
res7=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x7, Y: l7}))
print(res6)

[5 5 5 5 5 5 4 5 5 5 4 5 5 5 5 5 1 5 5 4 5 5 5 5 5 5 1 5 5 5 5 5 5 4]


In [48]:
confusion=[]
x1=np.sum(np.equal(res1, 0))
x2=np.sum(np.equal(res1, 1))
x3=np.sum(np.equal(res1, 2))
x4=np.sum(np.equal(res1, 3))
x5=np.sum(np.equal(res1, 4))
x6=np.sum(np.equal(res1, 5))
x7=np.sum(np.equal(res1, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res2, 0))
x2=np.sum(np.equal(res2, 1))
x3=np.sum(np.equal(res2, 2))
x4=np.sum(np.equal(res2, 3))
x5=np.sum(np.equal(res2, 4))
x6=np.sum(np.equal(res2, 5))
x7=np.sum(np.equal(res2, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res3, 0))
x2=np.sum(np.equal(res3, 1))
x3=np.sum(np.equal(res3, 2))
x4=np.sum(np.equal(res3, 3))
x5=np.sum(np.equal(res3, 4))
x6=np.sum(np.equal(res3, 5))
x7=np.sum(np.equal(res3, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res4, 0))
x2=np.sum(np.equal(res4, 1))
x3=np.sum(np.equal(res4, 2))
x4=np.sum(np.equal(res4, 3))
x5=np.sum(np.equal(res4, 4))
x6=np.sum(np.equal(res4, 5))
x7=np.sum(np.equal(res4, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res5, 0))
x2=np.sum(np.equal(res5, 1))
x3=np.sum(np.equal(res5, 2))
x4=np.sum(np.equal(res5, 3))
x5=np.sum(np.equal(res5, 4))
x6=np.sum(np.equal(res5, 5))
x7=np.sum(np.equal(res5, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res6, 0))
x2=np.sum(np.equal(res6, 1))
x3=np.sum(np.equal(res6, 2))
x4=np.sum(np.equal(res6, 3))
x5=np.sum(np.equal(res6, 4))
x6=np.sum(np.equal(res6, 5))
x7=np.sum(np.equal(res6, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res7, 0))
x2=np.sum(np.equal(res7, 1))
x3=np.sum(np.equal(res7, 2))
x4=np.sum(np.equal(res7, 3))
x5=np.sum(np.equal(res7, 4))
x6=np.sum(np.equal(res7, 5))
x7=np.sum(np.equal(res7, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

confusion=np.array(confusion)
print(confusion)

[[ 49   7   1   0   3   1   0]
 [  8 142   4   0   7  11   0]
 [ 13   5   8   0   6   0   1]
 [  1   7   1   0   1   0   0]
 [  4   7   7   0   9   0   0]
 [  0   2   0   0   4  28   0]
 [  0   5   2   0   3   1 198]]


In [44]:
# 70% train 30% test (NDVIclean)
# Train - 204
# Test - 87
X_train = data2[0:382,:]
X_test = data2[382:,:]
y_train = label[0:382]
y_test = label[382:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(382, 27)
(164, 27)
(382,)
(164,)


In [45]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(382, 7)
(164, 7)


In [49]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 20
num_batch = 10

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 1.35632257
Epoch: 2 cost = 1.33441043
Epoch: 3 cost = 1.31151867
Epoch: 4 cost = 1.28748078
Epoch: 5 cost = 1.26223249
Epoch: 6 cost = 1.23577595
Epoch: 7 cost = 1.20814018
Epoch: 8 cost = 1.17933741
Epoch: 9 cost = 1.14933047
Epoch: 10 cost = 1.11801891
Epoch: 11 cost = 1.08525419
Epoch: 12 cost = 1.05088167
Epoch: 13 cost = 1.01479449
Epoch: 14 cost = 0.97698884
Epoch: 15 cost = 0.93761692
Epoch: 16 cost = 0.89704447
Epoch: 17 cost = 0.85591197
Epoch: 18 cost = 0.81517215
Epoch: 19 cost = 0.77605901
Epoch: 20 cost = 0.73994613
Epoch: 21 cost = 0.70808160
Epoch: 22 cost = 0.68122032
Epoch: 23 cost = 0.65928390
Epoch: 24 cost = 0.64131792
Epoch: 25 cost = 0.62585948
Epoch: 26 cost = 0.61145496
Epoch: 27 cost = 0.59712119
Epoch: 28 cost = 0.58279064
Epoch: 29 cost = 0.56927991
Epoch: 30 cost = 0.55730522
Epoch: 31 cost = 0.54682674
Epoch: 32 cost = 0.53749449
Epoch: 33 cost = 0.52905748
Epoch: 34 cost = 0.52136536
Epoch: 35 cost = 0.51430914
Epoch: 36 cost = 0.50779104
E

Epoch: 316 cost = 0.00095520
Epoch: 317 cost = 0.00088199
Epoch: 318 cost = 0.00082360
Epoch: 319 cost = 0.00100346
Epoch: 320 cost = 0.00128824
Epoch: 321 cost = 0.00101695
Epoch: 322 cost = 0.00089311
Epoch: 323 cost = 0.00075700
Epoch: 324 cost = 0.00089726
Epoch: 325 cost = 0.00088230
Epoch: 326 cost = 0.00078177
Epoch: 327 cost = 0.00084754
Epoch: 328 cost = 0.00085578
Epoch: 329 cost = 0.00091965
Epoch: 330 cost = 0.00119646
Epoch: 331 cost = 0.00108131
Epoch: 332 cost = 0.00104619
Epoch: 333 cost = 0.00073046
Epoch: 334 cost = 0.00069588
Epoch: 335 cost = 0.00065689
Epoch: 336 cost = 0.00072954
Epoch: 337 cost = 0.00083205
Epoch: 338 cost = 0.00091200
Epoch: 339 cost = 0.00096367
Epoch: 340 cost = 0.00073706
Epoch: 341 cost = 0.00099353
Epoch: 342 cost = 0.00117353
Epoch: 343 cost = 0.00085359
Epoch: 344 cost = 0.00070537
Epoch: 345 cost = 0.00088119
Epoch: 346 cost = 0.00081440
Epoch: 347 cost = 0.00069005
Epoch: 348 cost = 0.00069993
Epoch: 349 cost = 0.00090929
Epoch: 350 cos

Epoch: 628 cost = 0.00052379
Epoch: 629 cost = 0.00046353
Epoch: 630 cost = 0.00059212
Epoch: 631 cost = 0.00049621
Epoch: 632 cost = 0.00032581
Epoch: 633 cost = 0.00048905
Epoch: 634 cost = 0.00048752
Epoch: 635 cost = 0.00035653
Epoch: 636 cost = 0.00034811
Epoch: 637 cost = 0.00029388
Epoch: 638 cost = 0.00034363
Epoch: 639 cost = 0.00050279
Epoch: 640 cost = 0.00052879
Epoch: 641 cost = 0.00053554
Epoch: 642 cost = 0.00059556
Epoch: 643 cost = 0.00046747
Epoch: 644 cost = 0.00039641
Epoch: 645 cost = 0.00038526
Epoch: 646 cost = 0.00042476
Epoch: 647 cost = 0.00036402
Epoch: 648 cost = 0.00045919
Epoch: 649 cost = 0.00037294
Epoch: 650 cost = 0.00033306
Epoch: 651 cost = 0.00046876
Epoch: 652 cost = 0.00057479
Epoch: 653 cost = 0.00037651
Epoch: 654 cost = 0.00066320
Epoch: 655 cost = 0.00058009
Epoch: 656 cost = 0.00032090
Epoch: 657 cost = 0.00031337
Epoch: 658 cost = 0.00028581
Epoch: 659 cost = 0.00025567
Epoch: 660 cost = 0.00035490
Epoch: 661 cost = 0.00051981
Epoch: 662 cos

Epoch: 941 cost = 0.00028961
Epoch: 942 cost = 0.00023239
Epoch: 943 cost = 0.00036601
Epoch: 944 cost = 0.00027506
Epoch: 945 cost = 0.00024850
Epoch: 946 cost = 0.00031988
Epoch: 947 cost = 0.00044052
Epoch: 948 cost = 0.00039093
Epoch: 949 cost = 0.00033858
Epoch: 950 cost = 0.00026584
Epoch: 951 cost = 0.00027281
Epoch: 952 cost = 0.00024237
Epoch: 953 cost = 0.00025706
Epoch: 954 cost = 0.00030881
Epoch: 955 cost = 0.00033053
Epoch: 956 cost = 0.00036624
Epoch: 957 cost = 0.00046276
Epoch: 958 cost = 0.00032864
Epoch: 959 cost = 0.00027823
Epoch: 960 cost = 0.00020451
Epoch: 961 cost = 0.00021003
Epoch: 962 cost = 0.00032144
Epoch: 963 cost = 0.00032496
Epoch: 964 cost = 0.00033112
Epoch: 965 cost = 0.00037990
Epoch: 966 cost = 0.00034423
Epoch: 967 cost = 0.00032431
Epoch: 968 cost = 0.00036891
Epoch: 969 cost = 0.00024518
Epoch: 970 cost = 0.00030636
Epoch: 971 cost = 0.00025502
Epoch: 972 cost = 0.00025319
Epoch: 973 cost = 0.00024812
Epoch: 974 cost = 0.00027014
Epoch: 975 cos

In [50]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.7621951
